In [4]:
#Importar bibliotecas
import numpy as np
from scipy.optimize import minimize, rosen, rosen_der
import matplotlib.pyplot as plt
import time


In [5]:
def minimizar(lista_geradores,restricao_geracao,demanda,lista_perdas):
    fun, cons = criar_lam(lista_geradores,demanda,lista_perdas)
    tuplas_chute_inicial = tuple(np.zeros(len(lista_geradores)))
    fun = eval(f"lambda x: {fun}")
    cons = ({'type': 'eq', 'fun':eval(f"lambda x: {cons}")})
    bnds = restricao_geracao
    res = minimize(fun, tuplas_chute_inicial, method='SLSQP', bounds=bnds,
               constraints=cons)
    return(res.x)





In [6]:
def criar_lam(lista_geradores,demanda,lista_perdas):
    lista_strings = [f"x[{i}]" for i in range(len(lista_geradores))]
    funcao_string = ''
    constraint = ''
    for x in range(len(lista_geradores)):
        funcao_string = funcao_string + '+'+str(lista_geradores[x][0])+'*'+lista_strings[x]+'**'+'2'+'+'+str(lista_geradores[x][1])+'*'+lista_strings[x]+'**'+'1'+'+'+str(lista_geradores[x][2])
        constraint = constraint +'+'+lista_strings[x]
        constraint=constraint + '-'+str(lista_perdas[x][0])+'*'+lista_strings[x]+'**'+'2'+'-'+str(lista_perdas[x][1])+'*'+lista_strings[x]+'**'+'1'+'-'+str(lista_perdas[x][2])
    constraint = constraint +'-'+str(demanda)
    return(funcao_string,constraint)

In [7]:
def custo_marginal(lista_geradores,restricao_geracao,demanda,lista_perdas):
    geracao = minimizar(lista_geradores,restricao_geracao,demanda,lista_perdas)
    vetor_derivada = np.arange(2,-1,-1)
    c_marginal = []
    for x in range(len(lista_geradores)):
        derivada = vetor_derivada*np.array(lista_geradores[x])
        derivada = derivada[:-1].copy()
        derivada_perdas = vetor_derivada*np.array(lista_perdas[x])
        derivada_perdas = derivada_perdas[:-1].copy()
        custo_mar_maq = np.polyval(derivada,geracao[x])
        custo_mar_perdas = np.polyval(derivada_perdas,geracao[x])
        c_marg_total = custo_mar_maq/(1-custo_mar_perdas)
        c_marginal.append(c_marg_total)
    return(c_marginal)


In [8]:
def perdas(lista_geradores,restricao_geracao,demanda,lista_perdas):
    vetor_perdas = []
    resultado = minimizar(lista_geradores,restricao_geracao,demanda,lista_perdas)
    for x in range(len(lista_perdas)):
        vetor_perdas.append(np.polyval(lista_perdas[x],resultado[x]))
    return(vetor_perdas)


In [9]:
def intervalo_demanda(lista_geradores,restricao,lista_demanda,lista_perdas):
    ti = time.time()
    vetor_intervalo = np.arange(lista_demanda[0],lista_demanda[1]+1,1)
    lista_custo_marginal = []
    for x in range(len(vetor_intervalo)):
        resultado =custo_marginal(lista_geradores,restricao,vetor_intervalo[x],lista_perdas)
        lista_custo_marginal.append(resultado)
    lista_custo_marginal = np.array(lista_custo_marginal)
    tf = time.time()
    tt = tf-ti
    return(lista_custo_marginal)


In [10]:
def i_demanda(lista_geradores,restricao_geracao,lista_demanda,lista_perdas):
    vetor_intervalo = np.arange(lista_demanda[0],lista_demanda[1]+1,1)
    lista_custo_marginal = []
    for x in range(len(vetor_intervalo)):
        resultado =custo_marginal(lista_geradores,restricao,vetor_intervalo[x],lista_perdas)


In [13]:
lista_geradores = [[0.04,7,0],[0.06,12,0],[0.02,9,0]]
restricao_geracao =((50,500),(50,200),(0,200))
lista_demanda=[390,510]
lista_perdas = [[0,0,0],[0,0,0],[0.001,0,0]]
demanda = 500

In [14]:
minimizar(lista_geradores,restricao_geracao,demanda,lista_perdas)

array([235.11530199, 115.07636662, 183.46932438])

In [15]:
perdas(lista_geradores,restricao_geracao,demanda,lista_perdas)

[0.0, 0.0, 33.66099298895016]

In [16]:
custo_marginal(lista_geradores,restricao_geracao,demanda,lista_perdas)

[25.809224159168572, 25.809163994143404, 25.809146212007143]

In [ ]:
def plotar_grafico(lista_geradores,restricao,lista_demanda,lista_perdas):
    vetor_intervalo = np.arange(lista_demanda[0],lista_demanda[1]+1,1)
    lista_nomes_geradores = [f"Gerador{i+1}" for i in range(len(lista_geradores))]
    lista = intervalo_demanda(lista_geradores,restricao,lista_demanda,lista_perdas)
    lista = lista.transpose()
    for x in range(len(lista)):
        plt.plot(vetor_intervalo, lista[x], label=lista_nomes_geradores[x])
    plt.xlabel("Demanda")
    plt.ylabel("Custo Marginal")
    plt.title("Custo_Marginal vs Demanda")
    plt.axvline(x=lista_demanda[0], color='red', linestyle='--')
    plt.legend()
    plt.show()




In [18]:
def encontrar_pontos2(lista_geradores,restricao_geracao,lista_demanda,lista_perdas):
    vetor_intervalo = np.arange(lista_demanda[0],lista_demanda[1]+1,1)
    #print(vetor_intervalo)
    lista_pd = []
    lista_check = []
    lista_comp = []
    lista = intervalo_demanda(lista_geradores,restricao_geracao,lista_demanda,lista_perdas)
    lista = np.round(lista,4)
    for x in range(len(lista)-1):
        lista_check.append([])
        for y in range(len(lista[x])):
            if lista[x+1][y] == lista[x][y]:
                lista_check[x].append('s')
            else:
                lista_check[x].append('n')
    for x in range(len(lista_check)-1):
        if lista_check[x+1] != lista_check[x]:
            lista_pd.append(vetor_intervalo[x]+1)
    return(lista_pd)




In [22]:
a = encontrar_pontos2(lista_geradores,restricao_geracao,lista_demanda,lista_perdas)
for x in range(len(a)):
  print(minimizar(lista_geradores,restricao_geracao,a[x],lista_perdas),a[x])
  print(minimizar(lista_geradores,restricao_geracao,a[x]-1,lista_perdas),a[x])
  print(minimizar(lista_geradores,restricao_geracao,a[x]-2,lista_perdas),a[x])
  print(minimizar(lista_geradores,restricao_geracao,a[x]+1,lista_perdas),a[x])
  print(minimizar(lista_geradores,restricao_geracao,a[x]+2,lista_perdas),a[x])

[ 70.  399.8] 460
[ 70.  398.8] 460
[ 70.  397.8] 460
[ 71.1145618 400.       ] 460
[ 72.51760651 400.        ] 460
[ 71.1145618 400.       ] 461
[ 70.  399.8] 461
[ 70.  398.8] 461
[ 72.51760651 400.        ] 461
[ 73.93183138 400.        ] 461


In [ ]:
for x in range(len(a)):
  print(custo_marginal(lista_geradores,restricao_geracao,a[x]-2,lista_perdas),a[x]-2)
  print(custo_marginal(lista_geradores,restricao_geracao,a[x]-1,lista_perdas),a[x]-1)
  print(custo_marginal(lista_geradores,restricao_geracao,a[x],lista_perdas),a[x])
  print(custo_marginal(lista_geradores,restricao_geracao,a[x]+1,lista_perdas),a[x]+1)
  print(custo_marginal(lista_geradores,restricao_geracao,a[x]+2,lista_perdas),a[x]+2)


In [16]:
def mudou_carga(lista_geradores,restricao_geracao,lista_demanda,lista_perdas):
    vetor_change = encontrar_pontos2
    print(vetor_change)
    lista = []
    for x in range(len(vetor_change)):
        lista.append([])
        for y in range(3):
          lista[x].append(minimizar(lista_geradores,restricao_geracao,vetor_change[x]+1,lista_perdas))
          lista[x].append(minimizar(lista_geradores,restricao_geracao,vetor_change[x]-1,lista_perdas))
          lista[x].append(minimizar(lista_geradores,restricao_geracao,vetor_change[x],lista_perdas))
          lista[x].append(vetor_change[x])
    return(lista)


In [ ]:
'''def encontrar_pontos(lista_geradores,restricao_geracao,lista_demanda,lista_perdas):
    vetor_intervalo = np.arange(lista_demanda[0],lista_demanda[1]+1,1)
    #print(vetor_intervalo)
    lista_pd = []
    lista_check = []
    lista_comp = []
    lista = intervalo_demanda(lista_geradores,restricao_geracao,lista_demanda,lista_perdas)
    lista = np.round(lista,2)
    for x in range(len(lista)-1):
        for y in range(len(lista[x])):
            if lista[x-1][y] == lista[x][y]:
                lista_check.append('s')
            else:
                lista_check.append('n')
            print(lista_check)
        if x == 0:
            if 's' in lista_check:
                lista_pd.append(vetor_intervalo[x])
        else:
            if lista_check != lista_comp:
                lista_pd.append(vetor_intervalo[x])
        print(lista_check,lista_comp)
    return(lista_pd)'''



